In [1]:
!git clone https://github.com/fannam/SoICT-LLM-Pruner.git

Cloning into 'SoICT-LLM-Pruner'...
remote: Enumerating objects: 143, done.
remote: Counting objects: 100% (143/143), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 143 (delta 65), reused 120 (delta 42), pack-reused 0 (from 0)
Receiving objects: 100% (143/143), 341.76 KiB | 1.37 MiB/s, done.
Resolving deltas: 100% (65/65), done.


In [2]:
%cd /content/SoICT-LLM-Pruner

/content/SoICT-LLM-Pruner


In [3]:
!pip install -e .

Obtaining file:///content/SoICT-LLM-Pruner
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Unins

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("TheGardener/Llama-3.2-1B-wikitext-finetune")
tokenizer = AutoTokenizer.from_pretrained("TheGardener/Llama-3.2-1B-wikitext-finetune")

In [7]:
!rm -rf ~/.cache/huggingface/datasets/eleuther_ai___wikitext_document_level
!pip install --upgrade datasets fsspec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 21.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


# ELEMENT

In [3]:
from datasets import load_dataset
from transformers import AutoTokenizer
from torch.utils.data import DataLoader

# 1) Load the document-level split
model_name = "TheGardener/Llama-3.2-1B-wikitext-finetune"

raw = load_dataset("EleutherAI/wikitext_document_level", "wikitext-103-raw-v1", split="train")

# 2) Filter out too-short docs (optional but recommended)
raw = raw.filter(lambda ex: len(ex["page"].strip()) > 256)

# 3) Shuffle & pick exactly 1024 documents
raw = raw.shuffle(seed=13).select(range(1024))

# 4) Tokenizer setup
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# 5) Tokenize to ~512 tokens each, pad/truncate
def tokenize_fn(examples):
    return tokenizer(
        examples["page"],
        truncation=True,
        max_length=512,
        padding="max_length"
    )
tok = raw.map(tokenize_fn, batched=True, remove_columns=["page"])

# 6) PyTorch formatting + DataLoader
tok.set_format(type="torch", columns=["input_ids", "attention_mask"])
calib_loader = DataLoader(tok, batch_size=4, shuffle=False)

In [ ]:
from estimator.element_estimator import Llama3ActivationElementEstimator

element_estimator = Llama3ActivationElementEstimator(model=model, device='cuda')

In [ ]:
mlp_importance = element_estimator.estimate_mlp_neurons(calib_loader)

Estimating MLP neuron importance:   0%|          | 0/256 [00:00<?, ?it/s]

In [ ]:
mlp_importance

{0: tensor([0.0003, 0.0003, 0.0003,  ..., 0.0003, 0.0003, 0.0003]),
 1: tensor([0.0003, 0.0005, 0.0004,  ..., 0.0003, 0.0014, 0.0006]),
 2: tensor([0.0005, 0.0004, 0.0005,  ..., 0.0004, 0.0005, 0.0004]),
 3: tensor([0.0008, 0.0011, 0.0006,  ..., 0.0007, 0.0005, 0.0014]),
 4: tensor([0.0010, 0.0006, 0.0010,  ..., 0.0007, 0.0006, 0.0013]),
 5: tensor([0.0010, 0.0009, 0.0024,  ..., 0.0009, 0.0010, 0.0008]),
 6: tensor([0.0007, 0.0015, 0.0008,  ..., 0.0006, 0.0006, 0.0013]),
 7: tensor([0.0012, 0.0007, 0.0010,  ..., 0.0016, 0.0009, 0.0009]),
 8: tensor([0.0014, 0.0011, 0.0008,  ..., 0.0007, 0.0009, 0.0006]),
 9: tensor([0.0013, 0.0007, 0.0010,  ..., 0.0010, 0.0012, 0.0009]),
 10: tensor([0.0011, 0.0060, 0.0017,  ..., 0.0010, 0.0017, 0.0026]),
 11: tensor([0.0013, 0.0011, 0.0013,  ..., 0.0015, 0.0010, 0.0013]),
 12: tensor([0.0012, 0.0011, 0.0011,  ..., 0.0011, 0.0012, 0.0012]),
 13: tensor([0.0015, 0.0014, 0.0046,  ..., 0.0017, 0.0016, 0.0017]),
 14: tensor([0.0019, 0.0019, 0.0016,  ..., 0

In [ ]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048, padding_idx=128001)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-0

In [ ]:
from pruner.element_level_pruner.ElementPruner import Llama3ElementPruner

element_pruner = Llama3ElementPruner(original_model=model, device='cuda')
pruned_model = element_pruner.prune_mlp(neuron_importance=mlp_importance, target_num_neurons=6144)

Checking size...
Pruning...
Pruned successfully


In [ ]:
pruned_model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048, padding_idx=128001)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=6144, bias=False)
          (up_proj): Linear(in_features=2048, out_features=6144, bias=False)
          (down_proj): Linear(in_features=6144, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-0

In [ ]:
prompt = "Paris is the capital of"

# Tokenize the input
input_ids = tokenizer.encode(prompt, return_tensors="pt").to('cuda:0')

# Generate text
output_ids = pruned_model.generate(
    input_ids,
    max_new_tokens=128,
    do_sample=True,
    temperature=0.7,
    top_k=50,
    top_p=0.95,
    num_return_sequences=1
)

# Decode and print the result
generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [ ]:
generated_text

'Paris is the capital of France and the city with the highest population in Europe. Located at the north of the city, the city of Paris is a major urban centre, with a population of over 2 million people. It is a part of the Paris metropolitan area, which has a population of 12 million. The city is the largest in France and the third largest in Europe. Paris has been a major center of culture, commerce, and politics for centuries. The Paris area is home to the headquarters of the European Central Bank and the United Nations. The city has a number of famous monuments, including the Eiffel Tower, the Arc de Triumph'

# LAYER

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("TheGardener/Llama-3.2-1B-wikitext-finetune")
tokenizer = AutoTokenizer.from_pretrained("TheGardener/Llama-3.2-1B-wikitext-finetune")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [2]:
from estimator.layer_estimator import Llama3SimilarityLayerEstimator

estimator = Llama3SimilarityLayerEstimator(model=model, device='cuda')

In [4]:
layer_importance = estimator.estimate(calib_loader)

Estimating layer importance:   0%|          | 0/256 [00:00<?, ?it/s]

In [5]:
layer_importance

{'attention': [0.2840938866138458,
  0.1868482381105423,
  0.1165439561009407,
  0.1341112107038498,
  0.12410645931959152,
  0.11546716094017029,
  0.12963491678237915,
  0.11570942401885986,
  0.12528440356254578,
  0.08994755893945694,
  0.04435575753450394,
  0.03240351006388664,
  0.03165782243013382,
  0.025978507474064827,
  0.04187101498246193,
  0.09118335694074631],
 'mlp': [0.43446943163871765,
  0.24559235572814941,
  0.2770686447620392,
  0.2687416672706604,
  0.2504979074001312,
  0.24702182412147522,
  0.21444520354270935,
  0.21305325627326965,
  0.2217458337545395,
  0.19544418156147003,
  0.1815342903137207,
  0.14314864575862885,
  0.11112510412931442,
  0.12379824370145798,
  0.13526363670825958,
  0.39146220684051514]}

In [6]:
from pruner.layer_level_pruner.LayerPruner import Llama3LayerPruner

layer_pruner = Llama3LayerPruner(model=model, device='cuda')

pruned_model = layer_pruner.prune(importance_scores=layer_importance, prune_counts={"attention": 2, "mlp": 1})

Pruned attention layers: {12, 13}
Pruned MLP layers: {12}


In [7]:
prompt = "Paris is the capital of"

# Tokenize the input
input_ids = tokenizer.encode(prompt, return_tensors="pt").to('cuda:0')

# Generate text
output_ids = pruned_model.generate(
    input_ids,
    max_new_tokens=128,
    do_sample=True,
    temperature=0.7,
    top_k=50,
    top_p=0.95,
    num_return_sequences=1
)

# Decode and print the result
generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [8]:
generated_text

'Paris is the capital of France and seat of government from which other French-speaking regions of western Europe also governments themselves derive their authority. Paris also has many other roles besides being government headquarters: it is also a city of arts and/or sports or both thereof respectively; it has numerous notable museums and museums galleries alike; it is also famous for its nightlife nightlife thereof respectively; it also has numerous notable museums and museums galleries alike; it also has numerous notable museums and/or museums alike; it also has numerous notable museums and/or museums alike; it also has numerous notable museums and/or museums alike; it also has numerous notable museums and/or museums alike; it also has numerous notable'

# BLOCK

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("TheGardener/Llama-3.2-1B-wikitext-finetune")
tokenizer = AutoTokenizer.from_pretrained("TheGardener/Llama-3.2-1B-wikitext-finetune")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/863 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/180 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.8k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/449 [00:00<?, ?B/s]

In [6]:
from estimator.block_estimator import Llama3SimilarityBlockEstimator

estimator = Llama3SimilarityBlockEstimator(model=model, block_size=1, device='cuda')

In [9]:
block_importance = estimator.estimate(calib_loader)

Estimating block size=1:   0%|          | 0/256 [00:00<?, ?it/s]

In [10]:
block_importance

[0.6334466051775962,
 0.3310318111907691,
 0.29434112226590514,
 0.3036053127143532,
 0.30152749363332987,
 0.29234713735058904,
 0.25770161021500826,
 0.2446600056719035,
 0.276974841253832,
 0.2462709485553205,
 0.19906081026419997,
 0.16367557691410184,
 0.1291869948618114,
 0.1363676143810153,
 0.14050430827774107,
 0.40826039435341954]

In [11]:
from pruner.block_level_pruner.BlockPruner import Llama3BlockPruner, Qwen2BlockPruner

block_pruner = Llama3BlockPruner(original_model=model, device='cuda')

pruned_model = block_pruner.prune(block_importance=block_importance, num_block_to_prune=4)

In [12]:
pruned_model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048, padding_idx=128001)
    (layers): ModuleList(
      (0-11): 12 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-0

In [13]:
prompt = "Paris is the capital of"

# Tokenize the input
input_ids = tokenizer.encode(prompt, return_tensors="pt").to('cuda:0')

# Generate text
output_ids = pruned_model.generate(
    input_ids,
    max_new_tokens=128,
    do_sample=True,
    temperature=0.7,
    top_k=50,
    top_p=0.95,
    num_return_sequences=1
)

# Decode and print the result
generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [14]:
generated_text

'Paris is the capital of capital in of of of of of of of of of of &yt-yt & ут- & ут-put- & ут- & ут- & ут- &zt- &� overt and & ov el and & ov and and and and and and and and and and and and and and and and and and hard &elt-elt- & ут- & ут- & ут- &üt- &� hot- &elt- & int-elt- & over and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and and'